In [ ]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.vector_ar.var_model import VAR
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [ ]:
# Загрузка данных
df = pd.read_csv('pollution.csv')
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

# Преобразование категориальных данных в числовые
df = pd.get_dummies(df)

# Визуализация данных
df['pollution'].plot()
plt.show()

In [ ]:
# Тест Дики-Фуллера на стационарность
result = adfuller(df['pollution'])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
# Проверка стационарности
if result[1] < 0.05:
    print("Ряд является стационарным")
else:
    print("Ряд не является стационарным")

In [ ]:
# VAR
model = VAR(df)
model_fit = model.fit()
prediction = model_fit.forecast(model_fit.y, steps=1)
print('VAR prediction:', prediction)

In [ ]:
# LSTM
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df)

train_length = int(len(scaled_data) * 0.8)
test_length = len(scaled_data) - train_length
train_data = scaled_data[0:train_length,:]
test_data = scaled_data[train_length:len(scaled_data),:]

In [ ]:
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), :]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

In [ ]:
look_back = 1
X_train, Y_train = create_dataset(train_data, look_back)
X_test, Y_test = create_dataset(test_data, look_back)

In [ ]:
# Архитектура LSTM
model = Sequential()
model.add(LSTM(100, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
# Обучение
model.fit(X_train, Y_train, epochs=20, batch_size=70, verbose=2)

In [ ]:
# Предсказание
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

In [ ]:
# Обратное преобразование для масштабирования
train_predict = scaler.inverse_transform(train_predict)
Y_train = scaler.inverse_transform([Y_train])
test_predict = scaler.inverse_transform(test_predict)
Y_test = scaler.inverse_transform([Y_test])

In [ ]:
# Визуализация результатов
plt.figure(figsize=(8,4))
plt.plot(df['pollution'], color='blue', label='Фактический уровень загрязнения')
plt.plot(range(len(Y_train[0])+look_back,len(Y_train[0])+2*look_back), test_predict, color='red', label='Предсказанный уровень загрязнения')
plt.title('Прогноз уровня загрязнения')
plt.xlabel('Дата')
plt.ylabel('Уровень загрязнения')
plt.legend()
plt.show()